<h1>Исследование надёжности заёмщиков</h1>

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**План работы**: сначала мы изучим общую информацию о полученных данных, далее - проведем предобработку данных, ответим на вопросы, являющиеся ключевыми в данном и исследовании и, наконец, напишем общий вывод на основании полученных в процессе работы результатов. 

<h3> Оглавление</h3>

1. [Шаг 1: Первичный обзор данных, получение общей информации о данных](#start)   
2. [Шаг 2: Предобработка данных](#preproc)   
    * [Работа с пропущенными значениями](#skipped)   
    * [Изменение типов данных](#changed)   
    * [Поиск и удаление дубликатов](#dubled)   
    * [Лемматизация](#lemmas)   
    * [Категоризация](#categories)   
3. [Шаг 3: Ответы на вопросы](#questions)
    * [Зависимость между наличием детей и возвратом кредита в срок](#q1)
    * [Зависимость между семейным положением и возвратом кредита в срок](#q2)
    * [Зависимость между уровнем дохода и возвратом кредита в срок](#q3)
    * [Зависимость между целью кредита и возратом его в срок](#q4)
4. [Шаг 4: Общие выводы](#sum)

<h3> Шаг 1: Первичный обзор данных, получение общей информации о данных </h3>
<a id='start'></a>

Импортируем необходимые нам для работы библиотеки, сохраним таблицу из файла в переменную и вызовем метод info() для просмотра общей информации о полученных нами данных. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
borrowers = pd.read_csv('/datasets/data.csv')
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Получим первые 10 записей таблицы для ознакомления с данными. 

In [3]:
borrowers.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод по данному Шагу**: просмотрев первые десять записей нашей таблицы, мы можем сформировать первое мнение: невооруженным взглядом заметны некорректные значения в одном из столбцов, а также проблемы с регистром категориальных переменных в другом. Также результат работы метода info() показывает нам, что в некоторых столбцах значений меньше чем в остальных: в таблице присутствуют пропущенные значения. Мы сможем обдумать причины их появления, являются ли они случайными или нет при более подробном изучении их природы: на следующем Шаге нашей работы. Также разберемся с другими проблемами, которые при первичном знакомстве с данными выявить не так просто.

<h3> Шаг 2: Предобработка данных </h3>
<a id='preproc'></a>

<h3> Работа с пропущенными значениями </h3>
<a id='skipped'></a>

Как известно, пропуски бывают разных видов. Сначала будем работать над пропусками в виде специальных значений (например, NaN), затем - над символами, знаменующими отсутствие данных в ячейке (например, '-'). 

Метод info(), который мы использовали на предыдущем шаге, показал нам, что в столбцах days_employed и total_income есть пропущенные значения. Обработаем их.

Просмотрим верхние 10 записей таблицы, в которых в столбце days_employed стоит NaN, на предмет наличия закономерностей между отсутствующими значениями и остальными данными. 

In [4]:
borrowers[borrowers['days_employed'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Можно предположить, что существует взаимосвязь между столбцом days_employed и столбцом total_income - значения отсутствуют и там, и там. Проверим данную догадку. Посмотрим, существуют ли записи таблицы, где отсутствует значение только в одном из столбцов.

In [5]:
borrowers[(borrowers['days_employed'].notna()) & (borrowers['total_income'].isna())].shape[0]

0

In [6]:
borrowers[(borrowers['days_employed'].isna()) & (borrowers['total_income'].notna())].shape[0]

0

Наша догадка верна - значения либо пропущены в обоих столбцах, либо ни в одном из них. Стаж работы и заработная плата - информация, без предоставления которой банки не выдают (в большинстве случаев) кредиты; это означает, что данные существуют, но в таблицу их не внесли. Возможно, сотрудники банка потеряли какие-то справки, предоставленные заемщиками. Или, если процесс заполнения таблицы был автоматизирован (вероятнее всего), система по какой-то причине не может получить доступ к данной информации. Может быть, следует сообщить об этом в банк. Также подтвержение нашей догадки влечет за собой вывод о том, что данные значения пропущены случайно, ибо, как было обозначено, данные существуют, они безусловно важны для исследования, но в таблицу не внесены.

Вернемся к более важному вопросу - что делать с пропущенными в этих столбцах значениями? Можно заполнить их нулями, но это будет некорректно по очевидным причинам. 
Может быть, их можно просто удалить? Посчитаем процент строк с пропущенными значениями в датафрейме.  

In [7]:
'{:.1%}'.format(borrowers['days_employed'].isna().sum() / borrowers.shape[0])

'10.1%'

Итак, строки с пропущенными значениями составляют 10,1 % от всех строк таблицы. Удаление такой части данных может привести к потере важных сведений о таблице и снизить точность исследования.

Оптимальный вариант - заполнить ячейки с пропущенными значениями средними значениями или медианами. Но посчитать эти значения для всей таблицы и использовать их было бы не совсем правильно. 
Мы вычислим среднее значение стажа для каждой возрастной группы и будем использовать его. Почему среднее, а не медиана? Потому что вероятность того, что в столбце будут присутствовать сильно отличающиеся значения, невелика - в большинстве своем люди начинают работать в примерно одинаковое время. 

Что же с колонкой total_income?

Для заполнения пропущенных значений в столбце с месячной заработной платой необходим более сложный подход; доход зависит от множества факторов - чтобы заполнить ячейки наиболее корректно требуется учесть максимальное их количество: стаж, образование и тип занятости. Обратимся к некоторой информации о таблице, чтобы понять, целесообразно ли заполнение пропущенных значений в столбце total_income.

In [8]:
borrowers['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [9]:
borrowers['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Итак, в столбце income_type у нас 8 уникальных значений, а в столбце education - 5 без учета регистра. Опираясь только на эти колонки, уже необходимо 40 (!) разных категорий. Можно поступить умнее - например, опустить студентов с ученой степенью (невозможно), не рассматривать безработных с разными типами образования, но все равно - количество групп окажется ощутимым. К тому же, в некоторых типах занятости большую роль играет еще и стаж. 

Но можно поступить еще умнее, и определить фактор, от которого сильнее всего зависит уровень дохода, и распределить заемщиков по группам по этому фактору, а затем посчитать медианный доход для каждой и заполнить пустые ячейки этими значениями. Будем использовать медианы, поскольку в таких вопросах, как доход, существуют значения, оторванные (сильно выше среднего) от остальных.

Примем точку зрения, что больше всего на уровень дохода влияет занимаемая должность. Поэтому сгруппируем таблицу по столбцу income_type и посчитаем медиану по total_income в каждой группе. 

In [10]:
 borrowers_group0 = borrowers.groupby('income_type')['total_income'].median()

In [11]:
 borrowers_group0

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Заменим NaN в столбце total_income полученными медианными значениями.

In [12]:
for i in borrowers['income_type'].unique():
    borrowers[borrowers['income_type'] == i] = borrowers[borrowers['income_type'] == i].fillna(value = {'total_income': borrowers_group0[i]})

Посмотрим, остались ли пропущенные значения в данном столбце.

In [13]:
borrowers[borrowers['total_income'].isna()].shape[0]

0

Пропущенных значений не осталось, мы заполнили все пустые ячейки.

Итак, теперь необходимо заполнить пустые ячейки в столбце days_employed.

Очевидно, для заполнения пропущенных значений в столбце days_employed потребуются остальные значения данного столбца и столбец с возрастом заемщиков (dob_years). Второй момент - ячейки и там, и там должны содержать в себе корректные данные. Ибо, в противном случае, мы и получим неверные значения в пустые ячейки.

На Шаге 1 можно было заметить первые аномалии - столбец days_employed содержит в себе отрицательные значения, а также слишком большие значения (см. строку 4). Возможная причина появления минусов перед числами - неверное распознавание символов тире и дефиса. Примем эту точку зрения и заменим все значения на положительные. Слишком большие значения заменим на NaN, поскольку неочевидно, какая ошибка привела к именно такому значению - очевидный лишний ноль не наблюдается, записывать стаж в часах вряд ли бы кто-то стал. 

Для замены значений в столбце напишем функцию. 25000 дней - это стаж 69 лет.

In [14]:
def days_employed (days_employed):
    if days_employed < 0:
        return -days_employed
    elif days_employed > 25000:
        return  np.nan
    return days_employed

Проверим работу функции.

In [15]:
days_employed(25001)

nan

In [16]:
days_employed(-19)

19

In [17]:
days_employed(12000)

12000

Функция работает корректно. Применим ее к столбцу days_employed и выведем первые 5 строк обновленного датафрейма для просмотра результатов. Также оценим новый процент строк с пропущенным в этом столбце значением. 

In [18]:
borrowers['days_employed'] = borrowers['days_employed'].apply(days_employed)
borrowers.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,NaN,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [19]:
'{:.1%}'.format(borrowers['days_employed'].isna().sum() / borrowers.shape[0])

'26.1%'

Значения заменены на более корректные. Процент ячеек с NaN в столбце существенно вырос, но по-другому никак - стаж в 400 тысяч дней явно точности выводам не прибавит. 

Проверим числа в колонке dob_years. Просмотрим список уникальных значений. 

In [20]:
borrowers['dob_years'].unique()

array([42, 36, 33, 32, 53, 27, 43, 50, 35, 41, 40, 65, 54, 56, 26, 48, 24,
       21, 57, 67, 28, 63, 62, 47, 34, 68, 25, 31, 30, 20, 49, 37, 45, 61,
       64, 44, 52, 46, 23, 38, 39, 51,  0, 59, 29, 60, 55, 58, 71, 22, 73,
       66, 69, 19, 72, 70, 74, 75], dtype=int64)

Ничего необычного, кроме нуля. 

Посчитаем процент строк со значением 0 в столбце dob_years.

In [21]:
'{:.1%}'.format(borrowers[borrowers['dob_years'] == 0]['dob_years'].count() / borrowers.shape[0])

'0.5%'

Имеем: в 0,5 % записей вместо возраста стоит 0. Очевидно, что в 0 лет нельзя брать кредит. Значит - значение отсутствует. Восстановить искомые значения по данным остальной таблицы можно, но очень и очень приближенно (например, по размеру стажа). 0,5 % - немного. Удалим такие записи.

In [22]:
borrowers = borrowers.drop(borrowers[borrowers['dob_years']==0].index)

Сейчас посчитаем средний стаж для каждой возрастной группы. Напишем функцию, которая определяет в какую возрастную группу определить заемщика. 

In [23]:
def age_group(age):
    if  age <30:
        return 1
    elif  age >=30 and age <40:
        return 2
    elif  age >=40 and age <50:
        return 3
    elif  age >=50 and age <60:
        return 4
    elif  age >=60 and age <70:
        return 5
    else:
        return 6

Распределим заемщиков по группам. Номер группы занесем в специальный столбец helper, который поможет посчитать средние значения. 

In [24]:
borrowers['helper'] = borrowers['dob_years'].apply(age_group)

Сгруппируем по номеру возрастной группы и найдем среднее значение стажа для каждой.

In [25]:
borrowers_group1 = borrowers.groupby('helper')['days_employed'].mean()
borrowers_group1

helper
1    1209.928267
2    2026.411457
3    2733.227228
4    3260.836218
5    3839.177181
6    4226.808923
Name: days_employed, dtype: float64

Просмотрим полученные значения для групп с 1 по 6 - результаты соответствуют здравому смыслу: чем выше номер группы, тем выше стаж. Заменим NaN в столбце days_employed этими значениями.

In [26]:
for i in range(1,7):
    borrowers[borrowers['helper'] == i] = borrowers[borrowers['helper'] == i].fillna(value = {'days_employed': borrowers_group1[i]})

Удалим вспомогательный столбец.

In [27]:
borrowers = borrowers.drop(columns=['helper'])

Вызовем метод info(), чтобы посмотреть - остались ли еще пропущенные значения.

In [28]:
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21424 entries, 0 to 21524
Data columns (total 12 columns):
children            21424 non-null int64
days_employed       21424 non-null float64
dob_years           21424 non-null int64
education           21424 non-null object
education_id        21424 non-null int64
family_status       21424 non-null object
family_status_id    21424 non-null int64
gender              21424 non-null object
income_type         21424 non-null object
debt                21424 non-null int64
total_income        21424 non-null float64
purpose             21424 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Количество ячеек в каждом столбце одинаковое. Мы удалили все NaN.

В процессе работы над пропущенными значениями, обозначенными NaN-типом, мы рассмотрели на наличие пропусков столбцы days_employed и total_income: в первом мы их заменили средними значениями по группе, во втором - значением '-1'. Также по ходу были обнаружены нетипичные значения в столбце dob_years - 0, такие строки были удалены, поскольку их было немного, а пропущенные значения трудно восстановить. 

Для столбцов education и income_type был вызван метод unique(), результат работы которого не говорит о чем-то необычном (в данном ключе) в этих столбцах.

Вызовем данный метод для столбцов, которые никак не проверялись ранее и которые потребуются нам при дальнейшем исследовании, и рассмотрим результаты на предмет наличия в записях аномалий.

In [29]:
borrowers['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

Значения -1 и 20 вызывают вопросы. '-1' это определенно маркер того, что информация о количестве детей у заемщика отсутствует, списать это на неправильно распознанный дефис или тире мы не можем, поскольку тогда появились бы значения '-2' и '-3'.  Значение '20' это явно ошибка. Возможных причин ее появления множество. 

Посмотрим, какой процент таблицы занимают строки со значениями '-1' и '20' в колонке children.

In [30]:
'{:.1%}'.format(borrowers[(borrowers['children'] == -1) | (borrowers['children'] == 20)]['children'].count() / borrowers.shape[0])

'0.6%'

Процент таких записей небольшой, удалим строки. 

In [31]:
borrowers = borrowers.drop(borrowers[(borrowers['children'] == -1) | (borrowers['children'] == 20)].index)

Проверим, получилось ли.

In [32]:
borrowers['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

Работа с колонкой children на  данном этапе завершена. Столбцы, значениями которых мы еще не интересовались, это family_status и debt. Посмотрим, все ли в порядке - вызовем метод unique для каждого из них.

In [33]:
borrowers['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [34]:
borrowers['debt'].unique()

array([0, 1], dtype=int64)

Уникальные значения данных столбцов корректны. Маркеров пустых ячеек нет. Колонку gender проверять не будем - влияние пола человека на его способность закрыть кредит в данной работе не исследуется. Остался еще один столбец - purpose. Взглянем и на него. 

In [35]:
borrowers['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Чего и следовало ожидать - ничего необычного.

**Вывод под подпункту 'Работа с пропущенными значениями'**: итак, в первой части Шага 2 мы разобрались с пропущенными значениями; в двух столбцах у нас они были указаны в явном виде - как NaN - для колонки days_employed мы поместили в пустые ячейки средние значения по группе, предварительно проследив, чтоб данные, на основании которых вычислялось среднее, были корректны; для колонки total_income мы заменили их на значение -1, поскольку было доказано, что заменять их вычисленными значениями не целесообразно, а удалять нельзя. В двух колонках - dob_years и children мы нашли маркеры пропущенных значений, эти строки мы удалили, поскольку их количество относительно размера датафрейма невелико. Теперь мы можем приступить к следующему подпункту.

<h3> Замена типов данных </h3>
<a id='changed'></a>

Вызовем метод info() еще раз - обратим особое внимание на типы данных столбцов.

In [36]:
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21302 entries, 0 to 21524
Data columns (total 12 columns):
children            21302 non-null int64
days_employed       21302 non-null float64
dob_years           21302 non-null int64
education           21302 non-null object
education_id        21302 non-null int64
family_status       21302 non-null object
family_status_id    21302 non-null int64
gender              21302 non-null object
income_type         21302 non-null object
debt                21302 non-null int64
total_income        21302 non-null float64
purpose             21302 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.1+ MB


Можно заметить странную вещь - столбцы days_employed (стаж) и total_income (размер ЗП) хранят в себе данные типа float.

Поняв этот факт, можно задать, как минимум, два вопроса: как и зачем эти значения вычисляются настолько точно? Очевидно, что такая точность здесь не нужна, мы можем поменять тип данных для days_employed и total_income с float на int. Сделаем это.

In [37]:
borrowers['days_employed'] = borrowers['days_employed'].astype('int64')
borrowers['total_income'] = borrowers['total_income'].astype('int64')

Был использован метод astype() поскольку он выполняет как раз то, что требуется - меняет тип данных в столбце на тот, который мы пожелаем. 
Проверим, все ли получилось.

In [38]:
borrowers.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21302 entries, 0 to 21524
Data columns (total 12 columns):
children            21302 non-null int64
days_employed       21302 non-null int64
dob_years           21302 non-null int64
education           21302 non-null object
education_id        21302 non-null int64
family_status       21302 non-null object
family_status_id    21302 non-null int64
gender              21302 non-null object
income_type         21302 non-null object
debt                21302 non-null int64
total_income        21302 non-null int64
purpose             21302 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


Результат работы метода info() говорит нам о том, что смена типов данных прошла успешно.

**Вывод под подпункту 'Замена типов данных'**: итак, во второй части Шага 2 мы поменяли тип данных колонок days_employed и total_income с float на int. Теперь наша таблица содержит в себе только данные типов int и object.

<h3>Удаление дубликатов</h3>
<a id='dubled'></a>

Вот, наш следующий подпункт - работа с дубликатами. Необходимо найти их и удалить. В нашем случае будет вестись поиск полностью одинаковых строк, поскольку специфика таблицы предполагает, что частичное дублирование (например, совпадение по столбцу children) - это нормально. Вообще говоря, полное совпадение данных для реальных людей тоже может иметь место, но вероятность этого куда меньше, чем вероятность того, что сбор данных в таблицу мог происходить с некоторыми ошибками (минуса в столбце days_employed, которые мы уже исправили, - прямое тому доказательство).

Очевидно, для качественной работы методов, применяемых к дублям, нам необходимо привести значения в столбцах, содержащих строки-категориальные переменные, к одному регистру. Колонок с данными типа object у нас 5. Разберемся с каждым.

Начнем с education. Вызовем метод unique().

In [39]:
borrowers['education'].unique()

array(['высшее', 'среднее', 'Среднее', 'СРЕДНЕЕ', 'ВЫСШЕЕ',
       'неоконченное высшее', 'начальное', 'Высшее',
       'НЕОКОНЧЕННОЕ ВЫСШЕЕ', 'Неоконченное высшее', 'НАЧАЛЬНОЕ',
       'Начальное', 'Ученая степень', 'УЧЕНАЯ СТЕПЕНЬ', 'ученая степень'],
      dtype=object)

Весьма ожидаемо - наблюдаем одни и те же категориальные переменные, но написанные разными по высоте буквами. Приведем все к одному виду. Ячейки будут содержать строки в нижнем регистре.

In [40]:
borrowers['education'] = borrowers['education'].str.lower()

Проверим, получилось ли.

In [41]:
borrowers['education'].unique()

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

Все вышло как следует - теперь в столбце только 5 уникальных значений. Проделаем аналогичную работу с остальными столбцами, если работа метода unique() покажет, что это необходимо. 

In [42]:
borrowers['family_status'].unique()

array(['женат / замужем', 'гражданский брак', 'вдовец / вдова',
       'в разводе', 'Не женат / не замужем'], dtype=object)

In [43]:
borrowers['gender'].unique()

array(['F', 'M', 'XNA'], dtype=object)

In [44]:
borrowers['income_type'].unique()

array(['сотрудник', 'пенсионер', 'компаньон', 'госслужащий',
       'безработный', 'предприниматель', 'студент', 'в декрете'],
      dtype=object)

In [45]:
borrowers['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Оказалось, проблемы были только с колонкой education. Можем смело искать дублирующие строки. Вызовем методы duplicated и sum, чтобы посчитать, сколько повторяющихся строк в таблице. 

In [46]:
borrowers.duplicated().sum()

71

Итак, обнаружено 71 полностью идентичных записей. Удалим их и вызовем данные методы еще раз, чтобы убедиться в своем успехе. 

In [47]:
borrowers = borrowers.drop_duplicates().reset_index(drop=True)
borrowers.duplicated().sum()

0

Удаление дубликатов произошло.

Возможная причина появления дубликатов в нашей таблице - выгрузка данных с разных источников. Например, была таблица с должниками и общая таблица, нам предоставили все вместе, не проверив, вдруг, один заемщик есть и в той, и в другой. 

**Вывод под подпункту 'Удаление дубликатов'**: итак, мы выявили и удалили все дубликаты в нашей таблице. Для этого нам потребовалось провести некоторые дополнительные действия со столбцом education, а именно, сделать так, чтобы категории образований не повторяли друг друга.

<h3>Лемматизация</h3>
<a id='lemmas'></a>

Следующий пункт в нашей работе - лемматизация. Мы будем лемматизировать уникальные значения столбца purpose, а затем посчитаем частоту упоминаний основных терминов в этом наборе значений с помошью Counter и выделим основные цели кредитов. Полученные значения будут нужны нам при категоризации заемщиков по основным целям кредитов в следующем пункте. 

Лемматизируем каждое значение по очереди в цикле и добавим к изначально пустому списку lemmas_purposes.

In [48]:
from pymystem3 import Mystem
from collections import Counter
lemmas_purposes =[]
m = Mystem()
unique_purposes = borrowers['purpose'].unique()
for i in range(len(unique_purposes)):
    try:
        lemmas_purposes+= m.lemmatize(unique_purposes[i])
    except:
        print('Что-то пошло не так.')

In [49]:
lemmas_counted = Counter(lemmas_purposes)

Взглянем на полученный нами счетчик лемм в наборе.

In [50]:
lemmas_counted

Counter({'покупка': 10,
         ' ': 59,
         'жилье': 7,
         '\n': 38,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 1,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'подержанный': 1,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1})

Можно выделить следующие цели: действия с жильем/недвижимостью, действия с автомобилем, свадьба и образование. Вызовем метод unique для столбца и мысленно убедимся, что все цели подходят под одну из групп.

In [51]:
borrowers['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Все значения в списке подходят под одну из четырех категорий.

**Вывод под подпункту 'Лемматизация'**: итак, мы лемматизировали список уникальных значений и посчитали с помощью специальной структуры данных Counter количество упоминаний каждого термина в списке значений. Полученная информация позволила нам определить четыре основных цели кредита.

<h3>Категоризация данных</h3>
<a id='categories'></a>

Крайний пункт в Шаге 2 - категоризация данных. Здесь мы сделаем две вещи. Во-первых, категоризируем заемщиков по уровню дохода и цели кредита, сформулированной более обобщенно, эти действия необходимы для ответа на поставленные вопросы. Во-вторых, выделим из таблицы 'словари': у нашего датафрейма есть особенность - он содержит в себе пары столбцов, отвечающие следующему паттерну: стобец 1, содержащий описание словом, + стобец 2, содержащий то же самое описание, но цифрой, соответствующей слову из столбца 1. От таких неявных повторений необходимо избавиться: вынести в отдельный 'словарь' категорий значения столбца 1 и соответствующие им идентификаторы из столбца 2. Спойлер: один из таких столбцов будем удалять, чтобы данные занимали меньше памяти и были более удобны для просмотра и операций над ними на следующем Шаге. 

Начнем со 'словарей' категорий.

Посмотрим еще раз на наши данные - найдем пары столбцов, отвечающие описанному выше паттерну.

In [52]:
borrowers.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,3260,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


Можно выделить следующие пары: 1)education + education_id, 2)family_status + family_status_is.

Создадим словари уникальных значений для каждой пары. Каждому значению из столбца 1 встанет в соответствие значение из столбца 2. 

In [53]:
borr_dict_edu = borrowers[['education','education_id']]
borr_dict_edu = borr_dict_edu.drop_duplicates().reset_index(drop=True)

In [54]:
borr_dict_fam = borrowers[['family_status','family_status_id']]
borr_dict_fam = borr_dict_fam.drop_duplicates().reset_index(drop=True)

Итак, мы выделили и создали два словаря. Словарь уровней образования и словарь семейных положений. Теперь можем удалить из нашей таблицы столбцы education и family_status. Почему именно их? Потому что данные поля хранят значения типа object, а они: занимают больше места, дольше обрабатываются.

In [55]:
borrowers = borrowers.drop(columns=['education', 'family_status'])

Посмотрим, как выглядит наша таблица сейчас.

In [56]:
borrowers.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,3260,53,1,1,F,пенсионер,0,158616,сыграть свадьбу


Посмотрим также, как выглядят наши словари, все ли получилось.

In [57]:
borr_dict_edu

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


In [58]:
borr_dict_fam

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Приступим к категоризации, а именно - к категоризации по уровню дохода. 

Для категоризации нам необходимо определить уровни (пусть их будет 5). За нас это сделает функция qcut, которой мы в качестве аргумента передадим наш столбец и желаемое количество категорий - 5, результат работы qcut сохраним в новый столбец income_level, который будет хранить в себе категорию, к которой отнесен тот или иной заемщик по уровню дохода. Посмотрим, что получилось.

In [59]:
borrowers['income_level'] =  pd.qcut(borrowers['total_income'], 5)

In [60]:
borrowers.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_level
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,"(214546.0, 2265604.0]"
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,"(98527.0, 132140.0]"
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,"(132140.0, 161379.0]"
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,"(214546.0, 2265604.0]"
4,0,3260,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,"(132140.0, 161379.0]"


Для более удобного восприятия таблицы и дальнейшего визуального представления результатов создадим 'словарь' категорий дохода. Каждому интервалу соотнесем целое число и понятное название, вместо столбца income_level создадим столбец income_level_id, тип данных которого будет int, а не category, как у income_level.

In [61]:
borr_dict_income = pd.DataFrame(columns=['income_level_id', 'income_level_interval', 'income_level'], data=[
    [1, borrowers['income_level'].unique().sort_values()[0], 'низкий'],
    [2, borrowers['income_level'].unique().sort_values()[1], 'ниже среднего'],
    [3, borrowers['income_level'].unique().sort_values()[2], 'средний'], 
    [4, borrowers['income_level'].unique().sort_values()[3], 'выше среднего'],
    [5, borrowers['income_level'].unique().sort_values()[4], 'высокий']
])

Для замены столбца income_level с интервалами на income_level_id с целочисленными значениями напишем функцию и проверим ее работу. 

In [62]:
def income_level_id(category):
    if category == borrowers['income_level'].unique().sort_values()[0]:
        return 1
    elif category == borrowers['income_level'].unique().sort_values()[1]:
        return 2
    elif category == borrowers['income_level'].unique().sort_values()[2]:
        return 3
    elif category == borrowers['income_level'].unique().sort_values()[3]:
        return 4
    elif category == borrowers['income_level'].unique().sort_values()[4]:
        return 5
    else:
        return 0

In [63]:
income_level_id(borrowers['income_level'].unique().sort_values()[1])

2

In [64]:
income_level_id(borrowers['income_level'].unique().sort_values()[3])

4

In [65]:
income_level_id(0)

0

Заменим столбцы и посмотрим на результирующую таблицу.

In [66]:
borrowers['income_level_id'] = borrowers['income_level'].apply(income_level_id)
borrowers = borrowers.drop(columns=['income_level'])

In [67]:
borrowers.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_level_id
0,1,8437,42,0,0,F,сотрудник,0,253875,покупка жилья,5
1,1,4024,36,1,0,F,сотрудник,0,112080,приобретение автомобиля,2
2,0,5623,33,1,0,M,сотрудник,0,145885,покупка жилья,3
3,3,4124,32,1,0,M,сотрудник,0,267628,дополнительное образование,5
4,0,3260,53,1,1,F,пенсионер,0,158616,сыграть свадьбу,3


Убедимся, что в новом столбце только корректные значения. 

In [68]:
borrowers['income_level_id'].unique()

[5, 2, 3, 1, 4]
Categories (5, int64): [1 < 2 < 3 < 4 < 5]

Все в порядке, мы категоризировали наших заемщиков по уровню дохода. Теперь займемся категоризацией по цели кредита.

Напишем для этого функцию. Также создадим 'словарь' для категорий и их числовых обозначений. В функции будем использовать стемминг, а не лемматизацию, поскольку это сильно быстрее. Известно, конечно, что данный метод несовершенен, мы проверим работу нашего стеммера после категоризации. 

In [69]:
borr_dict_purpose = pd.DataFrame(columns=['purpose_id', 'purpose'], data = [
    [1, 'действия с жильем/недвижимостью'],
    [2, 'действия с автомобилем'],
    [3, 'проведение свадьбы'],
    [4, 'образование']
])

In [70]:
from nltk.stem import SnowballStemmer
russian_stemmer = SnowballStemmer('russian')
word1 = russian_stemmer.stem('недвижимость')
word2 = russian_stemmer.stem('жилье')
word3 = russian_stemmer.stem('автомобиль')
word4 = russian_stemmer.stem('свадьба')
word5 = russian_stemmer.stem('образование')
def category1(purpose):
    for word in purpose.split():
        if russian_stemmer.stem(word) == word1 or russian_stemmer.stem(word) == word2:
            return 1
        elif russian_stemmer.stem(word) == word3:
            return 2
        elif russian_stemmer.stem(word) == word4:
            return 3
        elif russian_stemmer.stem(word) == word5:
            return 4
    return np.nan

Проверим работу функции.

In [71]:
category1('купить недвижимость')

1

In [72]:
category1('получить образование')

4

In [73]:
category1('отметить свадьбу')

3

In [74]:
category1('купить автомобиль')

2

In [75]:
category1('записать собаку в парикмахерскую')

nan

Создадим новый столбец для хранения категории цели.

In [76]:
borrowers['purpose_id'] = borrowers['purpose'].apply(category1)

Если мы правильно рассмотрели наши цели, и стеммер сработал корректно, в столбце purpose_id не должно быть значения NaN. Посмотрим, так ли это.

In [77]:
borrowers[borrowers['purpose_id'].isna()].head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,income_level_id,purpose_id
36,0,176,33,1,4,M,сотрудник,0,138830,автомобили,3,NaN
50,0,3839,63,1,0,F,пенсионер,0,92342,автомобили,1,NaN
54,0,3480,27,0,1,F,сотрудник,0,92238,автомобили,1,NaN
71,0,3839,62,1,0,F,пенсионер,0,43929,автомобили,1,NaN
79,0,3278,27,1,4,M,компаньон,0,253681,автомобили,5,NaN


Наш стеммер сработал не совсем корректно. Цель 'автомобили' не определилась в свою категорию. Сделаем это сами. Это категория 2. После проверим еще раз. Также нам надо поменять тип данных столбца purpose_id, так как после хранения NaN он превратился во float.

In [78]:
borrowers[borrowers['purpose'] == 'автомобили'] = borrowers[borrowers['purpose'] == 'автомобили'].fillna(value={'purpose_id':2})

In [79]:
borrowers[borrowers['purpose_id'].isna()].shape[0]

0

In [80]:
borrowers['purpose_id'] = borrowers['purpose_id'].astype('int64')

Удалим столбец purpose, тк он семантически идентичен со столбцом purpose_id и посмотрим на конечный вид нашей таблицы.

In [81]:
borrowers = borrowers.drop(columns=['purpose'])
borrowers.head()

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,income_level_id,purpose_id
0,1,8437,42,0,0,F,сотрудник,0,253875,5,1
1,1,4024,36,1,0,F,сотрудник,0,112080,2,2
2,0,5623,33,1,0,M,сотрудник,0,145885,3,1
3,3,4124,32,1,0,M,сотрудник,0,267628,5,4
4,0,3260,53,1,1,F,пенсионер,0,158616,3,3


**Вывод под подпункту 'Категоризация данных'**: в данном подпункте мы выделили из нашей таблица два словаря категорий и создали их. Это позволило нам сделать нашу таблицу более компактной - удалить из нее два столбца. Хранить данные в таком виде удобнее.
Также в данном подпункте мы категоризировали клиентов по уровню дохода и цели на кредит. Это позволит нам ответить на поставленные вопросы на Шаге 3.

**Вывод по Шагу 2 'Предобработка данных'**: итак, на данном Шаге мы проделали колоссальную работу - обработали пропущенные значения, поменяли типы данных, нашли и удалили дубликаты, лемматизировали один столбец, выделили два словаря категорий и категоризировали заемщиков, данные о которых содержит наша таблица, по уровню дохода и по цели на кредит в обобщенном виде. Благодаря всем этим действиям у нас есть таблица, соддержащая корректные во всех отношениях данные: без ячеек, не соответствующих логике поля, без дубликатов, без семантически идентичных столбцов. 

<h3> Шаг 3: Ответы на вопросы </h3>
<a id='questions'></a>

<h3> Зависимость между наличием детей и возвратом кредита в срок </h3>
<a id='q1'></a>

Итак, в данном пункте мы будем отвечать на вопрос - Есть ли зависимость между наличием детей и возвратом кредита в срок?. Более того, мы рассмотрим, как количество детей влияет на этот показатель.

Сгруппируем нашу таблицу по столбцу children, содержащему значение числа детей у заемщика. Применим к столбцy debt метод sum(), а к столбцу family_status_id - count(). Так мы получим в одном столбце количество людей, имевших задолженность в этой группе, а во втором - общее количество людей в группе. Посмотрим, что получилось.

In [82]:
borrowers_children_stat = borrowers.groupby('children').agg({'debt': 'sum', 'children':'count'})

In [83]:
borrowers_children_stat

,debt,children
children,,
0,1058,14022
1,441,4792
2,194,2039
3,27,328
4,4,41
5,0,9


Удалим строку с индексом 5, поскольку в данной группе не было людей, имевших задолженность по кредиту. Значит процент должников в данной группе будет равен 0. 

In [84]:
 borrowers_children_stat = borrowers_children_stat.drop(5)

Добавим в таблицу новый столбец proc - доля заемщиков, имевших проблемы с возвратом кредита в срок, в данной группе.

In [85]:
borrowers_children_stat['proc'] = round(borrowers_children_stat['debt'] / borrowers_children_stat['children'],4)

Посчитаем среднюю долю должников по таблице. Будем сравнивать с этими значениями числа, которые получаем, исследуя наши группы. 

In [86]:
borrowers_medium = borrowers['debt'].sum() / borrowers['debt'].count()

Добавим в нашу таблицу еще два столбца руководствуясь следующими принципами. Если значение столбца proc относительно среднего значения больше 1, то мы записываем его в первом столбце, во втором ставим -1, если данное отношение меньше 1, то мы во второй столбец записываем обратное ему, а в первый -1. Такой логикой мы будем руководствоваться на протяжении ответов и на все остальные вопросы. Такой формат хранения результатов сравнения со средним позволяет удобнее делать выводы - главное не забывать, в каком столбце что с чем сравниваем (для этого будем давать столбцам говорящие названия).

Проведем данный процесс в два этапа: заполним оба столбца полностью, затем с помощью функции удалим неподходящие (меньше 1). Также напишем функцию для конвертации доли (столбец proc) в проценты. Обе функции потребуются в дальнейшем.

In [87]:
def deleter(value):
    if value < 1:
        return -1
    else:
        return value
def procenter(value):
    return str(round(value*100, 2)) + '%'

Проверим работу функций.

In [88]:
deleter(0.87)

-1

In [89]:
deleter(1.01)

1.01

In [90]:
procenter(0.085)

'8.5%'

In [91]:
borrowers_children_stat['larger_than_medium'] = round(borrowers_children_stat['proc'] / borrowers_medium,2).apply(deleter)
borrowers_children_stat['less_than_medium'] = round(borrowers_medium / borrowers_children_stat['proc'],2).apply(deleter)
borrowers_children_stat['proc'] = borrowers_children_stat['proc'].apply(procenter)

Посмотрим, что получилось.

In [92]:
borrowers_children_stat

,debt,children,proc,larger_than_medium,less_than_medium
children,,,,,
0,1058,14022,7.55%,-1.00,1.08
1,441,4792,9.2%,1.13,-1.00
2,194,2039,9.51%,1.17,-1.00
3,27,328,8.23%,1.01,-1.00
4,4,41,9.76%,1.20,-1.00


Можем сделать выводы.

Итак, процент должников среди бездетных заемщиков ниже, чем среди заемщиков с детьми. Так, доля людей, имевших задолженности по кредиту, в группе заемщиков без детей в 1,08 раз ниже среднего, а в группах, где у заемщиков есть ребенок/дети, эта доля выше среднего. Зависимость этого отношения от количества детей тоже наблюдается: среди заемщиков с тремя детьми доля должников почти равна средней по таблице, а вот среди родителей двоих - уже в 1,17 раз больше среднего. Среди заемщиков с одним ребенком доля должников в 1,13 раз выше среднего. 

**Вывод по вопросу**: зависимость между наличием детей и возратом кредита в срок существует. Заемщики с детьми чаще имеют задолженности по возврату кредитов.

<h3>Зависимость между семейным положением и возвратом кредита в срок</h3>
<a id='q2'></a>

В данном пункте мы исследуем - Есть ли зависимость между семейным положением и возвратом кредита в срок?.

Взглянем на наш словарь семейных положений.

In [93]:
borr_dict_fam

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Итак, у нас 5 категорий семейных положений. Сгруппируем нашу таблицу по столбцу family_status_id и применим к столбцy debt метод sum(), а к столбцу family_status_id - count(). Так мы получим в одном столбце количество людей, имевших задолженность в этой группе, а во втором - общее количество людей в группе.

In [94]:
borrowers_family_stat = borrowers.groupby('family_status_id').agg({'debt':'sum','family_status_id':'count'})

Посмотрим, что получилось.

In [95]:
borrowers_family_stat

,debt,family_status_id
family_status_id,,
0,923,12213
1,383,4113
2,62,946
3,84,1179
4,272,2780


Добавим к этой таблице три новых столбца: в первом будем хранить долю людей, имевших задолженность, в данной группе, для второго и третьего столбца сделаем порядок, как в таблице, содержащей статистику о заемщиках, разделенных по признакому наличия/отсутствия детей (предыдущий вопрос). Но перед этим поменяем названия столбцов. Это потребуется в дальнейшем.

In [96]:
borrowers_family_stat.set_axis(['debt','borrowers'], axis='columns', inplace=True)
borrowers_family_stat['proc'] = round(borrowers_family_stat['debt'] / borrowers_family_stat['borrowers'], 4)
borrowers_family_stat['larger_than_medium'] = round(borrowers_family_stat['proc'] / borrowers_medium, 2).apply(deleter)
borrowers_family_stat['less_than_medium'] = round(borrowers_medium / borrowers_family_stat['proc'],2).apply(deleter)
borrowers_family_stat['proc'] = borrowers_family_stat['proc'].apply(procenter)

Для большей наглядности полученную таблицу с результатами соединим со словарем, который содержит наименования для family_status_id. Отсортируем по столбцу, содержащему в себе процент, и посмотрим, что получилось.

In [97]:
borrowers_family_stat = borrowers_family_stat.merge(borr_dict_fam, on='family_status_id', how='left')

In [98]:
borrowers_family_stat.sort_values(by='proc', ascending=False)

,family_status_id,debt,borrowers,proc,larger_than_medium,less_than_medium,family_status
4,4,272,2780,9.78%,1.20,-1.00,Не женат / не замужем
1,1,383,4113,9.31%,1.15,-1.00,гражданский брак
0,0,923,12213,7.56%,-1.00,1.07,женат / замужем
3,3,84,1179,7.12%,-1.00,1.14,в разводе
2,2,62,946,6.55%,-1.00,1.24,вдовец / вдова


Итак, по полученной таблице мы можем сделать выводы: процент людей, имевших задолженность по кредиту, среди не связанных официальными узами брака заемщиков (группы 'не женат/не замужем' и 'гражданский брак') выше среднего в 1,2 и 1,15 раза соответственно (9,78% и 9,31% должников в каждой группе соответственно). Количество людей, имевших задолженность по кредиту, в группе 'женат/незамужем' близко к среднему - всего в 1,07 раза меньше среднего значения (7,56%). А вот доля заемщиков, имевших проблемы с возвращением кредита в срок и относящихся к категориям 'в разводе' и 'вдовец/вдова', самая низкая, относительно среднего - меньше в 1,14 и 1,24 соответственно (в процентном виде: 7,12% и 6,55%).

**Вывод по вопросу**: зависимость между семейным положением и возвратом кредита в срок есть, причем существенная. Так, вдовы и вдовцы куда реже не платят кредит в срок, чем люди, состоящие в гражданском браке, или чье семейное положение - 'не женат/не замужем'. 

<h3>Зависимость между уровнем дохода и возвратом кредита в срок</h3>
<a id='q3'></a>

В данном пункте мы ответим на вопрос - Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Сгруппируем нашу таблицу по столбцу income_level_id, как и в предыдущем случае - к сгруппированной структуре приненим функцию sum к столбцу debt и count к income_level_id. 

In [99]:
borrowers_income_stat = borrowers.groupby('income_level_id').agg({'debt': 'sum', 'income_level_id':'count'})

Посмотрим, что получилось.

In [100]:
borrowers_income_stat

,debt,income_level_id
income_level_id,,
1,343,4247
2,356,4246
3,371,4246
4,356,4246
5,298,4246


Добавим в borrowers_income_stat еще три столбца: первый - доля людей, имевших задолженности по кредиту, в каждой группе, второй и третий - в соответствии с логикой, описаной выше (доля/среднее, или среднее/доля, или -1). Перед этим поменяем названия столбцов.

In [101]:
borrowers_income_stat.set_axis(['debt','borrowers'], axis='columns', inplace=True)
borrowers_income_stat['proc'] = round(borrowers_income_stat['debt'] / borrowers_income_stat['borrowers'], 4)
borrowers_income_stat['larger_than_medium'] = round(borrowers_income_stat['proc'] / borrowers_medium, 2).apply(deleter)
borrowers_income_stat['less_than_medium'] = round(borrowers_medium / borrowers_income_stat['proc'], 2).apply(deleter)
borrowers_income_stat['proc'] = borrowers_income_stat['proc'].apply(procenter)

Соединим получившуюся таблицу со 'словарем' уровней дохода для большей наглядности. 

In [102]:
borrowers_income_stat.merge(borr_dict_income, on='income_level_id', how='left').sort_values(by='proc', ascending=True)

,income_level_id,debt,borrowers,proc,larger_than_medium,less_than_medium,income_level_interval,income_level
4,5,298,4246,7.02%,-1.00,1.16,"(214546.0, 2265604.0]",высокий
0,1,343,4247,8.08%,1.00,1.00,"(20666.999, 98527.0]",низкий
1,2,356,4246,8.38%,1.03,-1.00,"(98527.0, 132140.0]",ниже среднего
3,4,356,4246,8.38%,1.03,-1.00,"(161379.0, 214546.0]",выше среднего
2,3,371,4246,8.74%,1.08,-1.00,"(132140.0, 161379.0]",средний


Сделаем выводы по полученной таблице.

Итак, у нас получилось 5 уровней дохода. Как видно из таблицы, процент должников среди людей с уровнем дохода 'высокий' (от 215 т.р.) меньше всего - в 1,16 раза ниже среднего. Удивительно, но, среди людей с уровнем дохода 'низкий' (до 98,5 т.р.) доля заемщиков, имевших задолженность по кредиту, равна средней по таблице. Заемщики-должники среди людей с оставшимися тремя уровнями дохода ( от 98,5 т.р. до 215 т.р.) составляют долю выше среднего, особенно выделяется группа со 'средним' доходом (от 132 т.р. до 161 т.р.), там этот процент выше всего - выше в 1,08 среднего по таблице. Но вообще, разница между долей заемщиков-должников во всех группах (кроме группы с доходом 'высокий') и средним по таблице значением совсем небольшая.  

**Вывод по вопросу**: зависимость между уровнем дохода и возвратом кредита в срок есть, но весьма специфичная. Люди с высоким (выше 215 т.р.)  реже всего имеют задолженности по кредитам, а вот среди остальных заемщиков - уровень дохода на возврат кредита в срок практически не влияет. 

<h3>Зависимость между целью кредита и возвратом его в срок</h3>
<a id='q4'></a>

В данном пункте мы ответим на вопрос - Как разные цели кредита влияют на его возврат в срок?

Сделаем из нашей таблицы сводную таблицу с помощью метода pivot_table. Посмотрим, что вышло.

In [112]:
borrowers_purpose_stat = borrowers.pivot_table(index=['purpose_id'], values=['debt'], aggfunc = ['sum','count'])

In [113]:
borrowers_purpose_stat

,sum,count
,debt,debt
purpose_id,,
1,777,10704
2,397,4258
3,181,2299
4,369,3970


Поменяем названия столбцов, а также добавим три новых столбца: первый будет содержать долю заемщиков в группе, имевших проблемы с возвратом кредита в срок, второй и третий будут заполнены в соответствии с логикой, описанной выше ( отношение доля/среднее, или отношение среднее/доля, или -1). 

In [114]:
borrowers_purpose_stat.set_axis(['debt','borrowers'], axis='columns', inplace=True)
borrowers_purpose_stat['proc'] = round(borrowers_purpose_stat['debt'] / borrowers_purpose_stat['borrowers'], 4)
borrowers_purpose_stat['larger_than_medium'] = round(borrowers_purpose_stat['proc'] / borrowers_medium, 2).apply(deleter)
borrowers_purpose_stat['less_than_medium'] = round(borrowers_medium / borrowers_purpose_stat['proc'], 2).apply(deleter)
borrowers_purpose_stat['proc'] = borrowers_purpose_stat['proc'].apply(procenter)

Для большей наглядности соединим получившуюся таблицу с нашим словарем категорий.

In [115]:
borrowers_purpose_stat.merge(borr_dict_purpose, on='purpose_id', how='left').sort_values(by='proc', ascending=True)

,purpose_id,debt,borrowers,proc,larger_than_medium,less_than_medium,purpose
0,1,777,10704,7.26%,-1.00,1.12,действия с жильем/недвижимостью
2,3,181,2299,7.87%,-1.00,1.03,проведение свадьбы
3,4,369,3970,9.29%,1.14,-1.00,образование
1,2,397,4258,9.32%,1.15,-1.00,действия с автомобилем


Итак, мы можем сделать выводы по получившейся таблице.

Процент людей, имевших проблемы с возвратом кредита в срок, в группе заемщиков, бравших займ на действия с недвижимостью/жильем меньше всего (7,26%, что в 1,12 раз ниже среднего). А вот в группе людей, бравших кредит на свадьбу, такой процент практически идентичен среднему (всего в 1,03 раза меньше среднего). Почти одинаковая доля должников в группах с кредитными целями 'образование' и 'действия с автомобилем' - 9,29% и 9,32% соответственно (в 1,14 и 1,15 раз выше среднего).

**Вывод по вопросу**: цель кредита влияет на вероятность того, будет ли иметь заемщик проблемы с возвратом его в срок. Люди, берущие кредит на действия с недвижимостью/жильем, реже имеют задолженность, а берущие с целью 'образование' и 'действия с автомобилем' - чаще.

<h3>Шаг 4: Общие выводы</h3>
<a id='sum'></a>

Итак, мы ответили на 4 вопроса: разобрались, существует ли зависимость между возвратом кредита в срок и другими факторами: семейным положением, наличием детей, уровнем дохода и целью кредита. Чтобы подвести итог, сведем некоторые элементы созданных данее таблиц в одну таблицу.

Наша таблица будет содержать факторы, для каждого фактора: наибольшее значение того, во сколько раз процент заемщиков, имевших проблемы с возвратом кредита в срок, в одной из групп (при группировке именно по этому фактору) выше и ниже среднего (наибольшие значения столбцов larger_than_medium и less_than_medium соответственно).

In [116]:
borrowers_accum = pd.DataFrame(columns=['factor_name', 'larger_than_MAX', 'less_than_MAX'], data =[
    ['наличие детей', borrowers_children_stat['larger_than_medium'].max(), borrowers_children_stat['less_than_medium'].max()],
    ['семейное положение', borrowers_family_stat['larger_than_medium'].max(), borrowers_family_stat['less_than_medium'].max()],
    ['уровень дохода', borrowers_income_stat['larger_than_medium'].max(), borrowers_income_stat['less_than_medium'].max()],
    ['цель кредита', borrowers_purpose_stat['larger_than_medium'].max(), borrowers_purpose_stat['less_than_medium'].max()]
])

In [117]:
borrowers_accum

,factor_name,larger_than_MAX,less_than_MAX
0,наличие детей,1.20,1.08
1,семейное положение,1.20,1.24
2,уровень дохода,1.08,1.16
3,цель кредита,1.15,1.12


Сделаем вывод. Банк, вероятно, больше всего интересует, какие группы людей с большей вероятностью не вернут кредит в срок, так называемая - группа риска. Мы можем предоставить ответ в виде списка факторов, влияющих на это, по убыванию. На первом месте - на что нужно сильнее всего обращать внимание, на последнем - на что меньше всего. 

In [118]:
borrowers_accum.sort_values(by='larger_than_MAX', ascending=False)

,factor_name,larger_than_MAX,less_than_MAX
0,наличие детей,1.20,1.08
1,семейное положение,1.20,1.24
3,цель кредита,1.15,1.12
2,уровень дохода,1.08,1.16


Итак, первым делом следует обращать внимание на семейное положение и на наличие детей (на их количество), затем на цель кредита, и только потом на уровень дохода. Какие именно группы по каждому фактору входят в группу риска можно посмотреть выше, на Шаге 3.